In [1]:
import warnings
import pandas as pd
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

In [2]:
# Read CSV File
df = pd.read_csv(r'C:\Users\Rafael_Fagundes\Downloads\GCM_Browser_YTD.csv'
                 , skipfooter=1, skiprows=10, parse_dates=['Date'])

In [3]:
# Add Premutive Flag and Browser mapping
df['Permutive_flag'] = df['Placement'].str.contains(r'Permutive|Permutvie|permu|Permu|ermu'
                                                                  , case=False, regex=True)

browser_mapping = {
    'Chrome' : 'Chrome',
    'iPhone / iPod touch' : 'Safari',
    'Microsoft Edge' : 'Edge',
    'Android' : 'Chrome',
    'Unknown' : 'Other',
    'Safari' : 'Safari',
    'Firefox' : 'Firefox',
    'Mozilla' : 'Firefox',
    'iPad' : 'Safari',
    'Roku' : 'Other',
    'Opera' : 'Other',
    'Microsoft Internet Explorer' : 'Other',
    'Yandex' : 'Other',
    'Netscape Navigator' : 'Other',
    'Opera Next' : 'Other',
    '(not set)' : 'Other'
}

df['Browser Group'] = df['Browser/Platform'].map(browser_mapping)

In [4]:
# Group dataframe
df_grouped = df.groupby(['Date','Campaign','Permutive_flag','Browser Group'])['Impressions'].sum().reset_index()

df_grouped.head()

,Date,Campaign,Permutive_flag,Browser Group,Impressions
0,2023-02-04,2021_1H_US_MB_MF_SOCIAL_ALLPLATFORMS,False,Chrome,0
1,2023-02-04,2021_1H_US_MB_MF_SOCIAL_ALLPLATFORMS,False,Edge,0
2,2023-02-04,2021_2H_US_ISG_MF_DISPLAY_NA_PAN FF,False,Chrome,1
3,2023-02-04,2021_AO_FR_AW_AP_DISPLAY_NA_HEAVEN MEDIA,False,Chrome,0
4,2023-02-04,2021_Q1_FR_CON_UF_SOCIAL_FR CON,False,Other,0


Columns

In [5]:
Country_Code = '(_US_|_CA_|_UK_)'
BU_Code = '(_AW_|_B2B-CHANNEL_|_B2B-COM_|_B2B-CSG_|_B2B-ISG_|_B2B-MB_|_BRAND-PAN-DELL_|_CET_|_COM_|_CON_|_CONS_|_CSB_|_CSBO-AW_|_CSBO-AW/GSERIES_|_CSBO-CON_|_CSBO-CSB_|_CSBO-GSERIES_|_CSBO-OUT_|_CSBO-PAN-DELL_|_CSBO-PROGRAMS_|_CSBO-SB_|_CSG_|_DTC_|_GAM_|_ISG_|_ISG _|_MB_|_MPP_|_OUT_|_OUTCON_|_OUTSB_|_PANDELL_|_PAN-DELL_|_RET_|_RETAIL_|_SB_|_SBG-ISG_|_SBG-PROGRAMS_|_SBG-SB_|_SB-ISG_|_SB-SBG_|_SMB_)'
Funnel_Code = '(_AP_|_BSP_|_DCR_|_HP_|_LF_|_LG_|_MF_|_PP_|_UF_)'
FY_Code = '(2022_Q1_|2022_Q2_|2022_Q3_|2022_Q4_|2023_Q1_|2023_Q2_|2023_Q3_|2023_Q4_|2024_Q1_|2024_Q2_|2024_Q3_|2024_Q4_)'
Vehicle_Code = '(_AFF_|_AUDIO_|_BRAND_|_CTV_|_Direct_|_DISPLAY_|_DISPLAY-DCO_|_DISPLAY-MOBILE_|_EMAIL-BAU_|_EMAIL-LCM_|_LEAD GEN_|_LEAD-GEN_|_MOBILE_|_RET_|_SEM_|_SNS_|_SOCIAL_|_VIDEO_|_VIDEO-MOBILE_)'

df_grouped['Country'] = df_grouped['Campaign'].str.upper().str.extract(Country_Code)
df_grouped['BU'] = df_grouped['Campaign'].str.upper().str.extract(BU_Code)
df_grouped['Funnel'] = df_grouped['Campaign'].str.upper().str.extract(Funnel_Code)
#df_grouped['Fiscal Quarter'] = df_grouped['Campaign'].str.upper().str.extract(FY_Code)
df_grouped['Vehicle'] = df_grouped['Campaign'].str.upper().str.extract(Vehicle_Code)

df_grouped['Country'] = df_grouped['Country'].str.replace('_', '')
df_grouped['BU'] = df_grouped['BU'].str.replace('_', '')
df_grouped['Funnel'] = df_grouped['Funnel'].str.replace('_', '')
#df_grouped['Fiscal Quarter'] = df_grouped['Fiscal Quarter'].str.replace('_', '')
df_grouped['Vehicle'] = df_grouped['Vehicle'].str.replace('_', '')

df_grouped.head()

,Date,Campaign,Permutive_flag,Browser Group,Impressions,Country,BU,Funnel,Vehicle
0,2023-02-04,2021_1H_US_MB_MF_SOCIAL_ALLPLATFORMS,False,Chrome,0,US,MB,MF,SOCIAL
1,2023-02-04,2021_1H_US_MB_MF_SOCIAL_ALLPLATFORMS,False,Edge,0,US,MB,MF,SOCIAL
2,2023-02-04,2021_2H_US_ISG_MF_DISPLAY_NA_PAN FF,False,Chrome,1,US,ISG,MF,DISPLAY
3,2023-02-04,2021_AO_FR_AW_AP_DISPLAY_NA_HEAVEN MEDIA,False,Chrome,0,NaN,AW,AP,DISPLAY
4,2023-02-04,2021_Q1_FR_CON_UF_SOCIAL_FR CON,False,Other,0,NaN,CON,UF,SOCIAL


Filter

In [6]:
df_filtered = df_grouped[~df_grouped['Campaign'].isin(['DO NOT USE'])]

Calendar

In [7]:
df_calendar = pd.read_csv('../../Helper/Calendar.csv', parse_dates=['Date'])

df_calendar = df_calendar[['Date','Fiscal Quarter','Fiscal Week Quarter']]

Merge

In [8]:
df_merged = pd.merge(df_filtered, df_calendar, on='Date', how='left')

df_merged.head()

,Date,Campaign,Permutive_flag,Browser Group,Impressions,Country,BU,Funnel,Vehicle,Fiscal Quarter,Fiscal Week Quarter
0,2023-02-04,2021_1H_US_MB_MF_SOCIAL_ALLPLATFORMS,False,Chrome,0,US,MB,MF,SOCIAL,2024-Q1,2024-W01
1,2023-02-04,2021_1H_US_MB_MF_SOCIAL_ALLPLATFORMS,False,Edge,0,US,MB,MF,SOCIAL,2024-Q1,2024-W01
2,2023-02-04,2021_2H_US_ISG_MF_DISPLAY_NA_PAN FF,False,Chrome,1,US,ISG,MF,DISPLAY,2024-Q1,2024-W01
3,2023-02-04,2021_AO_FR_AW_AP_DISPLAY_NA_HEAVEN MEDIA,False,Chrome,0,NaN,AW,AP,DISPLAY,2024-Q1,2024-W01
4,2023-02-04,2021_Q1_FR_CON_UF_SOCIAL_FR CON,False,Other,0,NaN,CON,UF,SOCIAL,2024-Q1,2024-W01


In [12]:
df_regroup = df_merged.groupby(['Fiscal Quarter','Fiscal Week Quarter','Campaign','Permutive_flag'
                                ,'Browser Group','Country','BU','Funnel','Vehicle'])['Impressions'].sum().reset_index()

df_regroup.head()

,Fiscal Quarter,Fiscal Week Quarter,Campaign,Permutive_flag,Browser Group,Country,BU,Funnel,Vehicle,Impressions
0,2024-Q1,2024-W01,2021_1H_US_MB_MF_SOCIAL_ALLPLATFORMS,False,Chrome,US,MB,MF,SOCIAL,0
1,2024-Q1,2024-W01,2021_1H_US_MB_MF_SOCIAL_ALLPLATFORMS,False,Edge,US,MB,MF,SOCIAL,0
2,2024-Q1,2024-W01,2021_2H_US_ISG_MF_DISPLAY_NA_PAN FF,False,Chrome,US,ISG,MF,DISPLAY,10
3,2024-Q1,2024-W01,2021_2H_US_ISG_MF_DISPLAY_NA_PAN FF,False,Edge,US,ISG,MF,DISPLAY,1
4,2024-Q1,2024-W01,2021_2H_US_ISG_MF_DISPLAY_NA_PAN FF,False,Safari,US,ISG,MF,DISPLAY,3


In [13]:
df_regroup.to_csv('csv/Browser_Mix_by_Campaign.csv')

In [11]:
df_pivoted = df_grouped.pivot(index=['Date','Campaign', 'Permutive_flag'], columns='Browser Group', values='Impressions').reset_index()

df_pivoted.fillna(0,inplace=True)